# Expert Systems

**Author: Brian van den Berg**

In this Python Notebook, I will be creating an expert system to demonstrate how it can be used in practise. First we will need to import the library 'experta', which is supported by Python 3.8.

In [1]:
import experta
from experta import *
import sys

# Print experta library version
print("experta version:", experta.__version__)

# Print Python version
print("Python version:", sys.version)

experta version: 1.9.4
Python version: 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]


## Setting up Facts and Rules

To make an expert system work, we need to define the rules that it has to follow. This can be done in context of the initial facts set-up for the system. In our case, we will first define the people with their parents and we will let the expert system take care of inner relations using rules.

This expert system was set-up to be "future proof" by allowing a variable amount of parents and gender-neutral relation terms. In basis, the entire family tree has been abstracted to its core-components.

In [2]:
# Define a Fact class to represent individuals in the family tree
class Person(Fact):
    pass

# Define a KnowledgeEngine class for the family tree expert system
class FamilyTree(KnowledgeEngine):
    @DefFacts()
    def _initial_action(self):
        # Initial facts to represent individuals and relationships in the family tree

        # Person
        yield Person(name="Ethan", parents=["Noah", "Ava"])

        # Siblings
        yield Person(name="Olivia", parents=["Noah", "Ava"])
        yield Person(name="Mason", parents=["Noah", "Ava"])

        # Parents
        yield Person(name="Noah", parents=["Logan", "Sophia"])
        yield Person(name="Ava", parents=["Caleb", "Isabella"])

        # Grand-Parents
        yield Person(name="Logan", parents=["Jackson", "Grace"])
        yield Person(name="Sophia", parents=["Liam", "Emily"])
        yield Person(name="Caleb", parents=["Owen", "Aria"])
        yield Person(name="Isabella", parents=["Carter", "Lily"])

        # Piblings
        yield Person(name="Amelia", parents=["Logan", "Sophia"])
        yield Person(name="Dylan", parents=["Caleb", "Isabella"])

        # Cousins
        yield Person(name="Stella", parents=["Dylan", "Scarlett", "Harper"])
        yield Person(name="Sebastian", parents=["Dylan", "Scarlett", "Harper"])
        yield Person(name="Penelope", parents=["Dylan", "Scarlett", "Harper"])
        yield Person(name="Lucas", parents=["Amelia", "Gabriel"])

    # Rule to infer parent-child relationship
    @Rule(Person(name=MATCH.x, parents=MATCH.parents))
    def parent_rule(self, x, parents):
        for parent in parents:
            self.declare(Fact(name=x, parent=parent))
            self.declare(Fact(name=parent, child=x))

    # Rule to infer sibling relationship
    @Rule(Fact(name=MATCH.x, parent=MATCH.parent),
          Fact(name=MATCH.y, parent=MATCH.parent),
          TEST(lambda x, y: x != y))
    def sibling_rule(self, x, y):
        self.declare(Fact(name=x, sibling=y))

    # Rule to infer grandparent-grandchild relationship
    @Rule(Fact(name=MATCH.x, parent=MATCH.parent),
          Fact(name=MATCH.parent, parent=MATCH.y))
    def grandparent_rule(self, x, y):
        self.declare(Fact(name=x, grandparent=y))
        self.declare(Fact(name=y, grandchild=x))

    # Rule to infer pibling-niephling relationship
    @Rule(Fact(name=MATCH.x, grandparent=MATCH.grandparent),
          Fact(name=MATCH.y, parent=MATCH.grandparent))
    def pibling_rule(self, x, y):
        parents = self.get_parents(x)
        if y not in parents:
            self.declare(Fact(name=x, pibling=y))
            self.declare(Fact(name=y, niephling=x))

    # Rule to infer cousin relationships
    @Rule(Fact(name=MATCH.x, pibling=MATCH.pibling),
          Fact(name=MATCH.y, parent=MATCH.pibling))
    def cousin_rule(self, x, y):
        self.declare(Fact(name=x, cousin=y))

    # Function to fetch every parent of a person
    def get_parents(self, person_name):
        parents = set()
        for i in self.facts:
            fact = self.facts[i]
            if fact.get('name') == person_name:
                if 'parent' in fact:
                    parents.add(fact['parent'])
        return parents

    # Function to fetch every relation of a person
    def get_relations(self, person_name):
        results = {
            'parents': set(),
            'children': set(),
            'siblings': set(),
            'grandparents': set(),
            'grandchildren': set(),
            'piblings': set(),
            'niephlings': set(),
            'cousins': set()
        }
        for i in self.facts:
            fact = self.facts[i]
            if fact.get('name') == person_name:
                if 'parent' in fact:
                    results['parents'].add(fact['parent'])
                if 'child' in fact:
                    results['children'].add(fact['child'])
                if 'sibling' in fact:
                    results['siblings'].add(fact['sibling'])
                if 'grandparent' in fact:
                    results['grandparents'].add(fact['grandparent'])
                if 'grandchild' in fact:
                    results['grandchildren'].add(fact['grandchild'])
                if 'pibling' in fact:
                    results['piblings'].add(fact['pibling'])
                if 'niephling' in fact:
                    results['niephlings'].add(fact['niephling'])
                if 'cousin' in fact:
                    results['cousins'].add(fact['cousin'])
        return results
    
# Execution of the expert system
family_tree = FamilyTree()
family_tree.reset()
family_tree.run()
print(family_tree.facts)

<f-0>: InitialFact()
<f-1>: Person(name='Ethan', parents=frozenlist(['Noah', 'Ava']))
<f-2>: Person(name='Olivia', parents=frozenlist(['Noah', 'Ava']))
<f-3>: Person(name='Mason', parents=frozenlist(['Noah', 'Ava']))
<f-4>: Person(name='Noah', parents=frozenlist(['Logan', 'Sophia']))
<f-5>: Person(name='Ava', parents=frozenlist(['Caleb', 'Isabella']))
<f-6>: Person(name='Logan', parents=frozenlist(['Jackson', 'Grace']))
<f-7>: Person(name='Sophia', parents=frozenlist(['Liam', 'Emily']))
<f-8>: Person(name='Caleb', parents=frozenlist(['Owen', 'Aria']))
<f-9>: Person(name='Isabella', parents=frozenlist(['Carter', 'Lily']))
<f-10>: Person(name='Amelia', parents=frozenlist(['Logan', 'Sophia']))
<f-11>: Person(name='Dylan', parents=frozenlist(['Caleb', 'Isabella']))
<f-12>: Person(name='Stella', parents=frozenlist(['Dylan', 'Scarlett', 'Harper']))
<f-13>: Person(name='Sebastian', parents=frozenlist(['Dylan', 'Scarlett', 'Harper']))
<f-14>: Person(name='Penelope', parents=frozenlist(['Dylan'

## Usage of the System

To use the expert system, you would need to define functions that return specific relations. In my example, I made a get_parents and get_relations function. The get_parents function was needed to check wether a potential aunt/uncle relation was actually already a parent relation.

The get_relations function was created to check if the correct relations were assigned to every person in the system. This also serves as prove that the system works for this assignment and it will be used in the report surrounding expert systems.

In [3]:
# Execution of the expert system
family_tree = FamilyTree()
family_tree.reset()
family_tree.run()

# Usage example
person_name = "Ethan"
ethan_relations  = family_tree.get_relations(person_name)

# Printing the formatted output
print(f'Relations for {person_name}:')
print("- Parents:", ', '.join(ethan_relations['parents']) if ethan_relations['parents'] else 'None')
print("- Children:", ', '.join(ethan_relations['children']) if ethan_relations['children'] else 'None')
print("- Siblings:", ', '.join(ethan_relations['siblings']) if ethan_relations['siblings'] else 'None')
print("- Grandparents:", ', '.join(ethan_relations['grandparents']) if ethan_relations['grandparents'] else 'None')
print("- Grandchildren:", ', '.join(ethan_relations['grandchildren']) if ethan_relations['grandchildren'] else 'None')
print("- Piblings:", ', '.join(ethan_relations['piblings']) if ethan_relations['piblings'] else 'None')
print("- Niephlings:", ', '.join(ethan_relations['niephlings']) if ethan_relations['niephlings'] else 'None')
print("- Cousins:", ', '.join(ethan_relations['cousins']) if ethan_relations['cousins'] else 'None')

Relations for Ethan:
- Parents: Ava, Noah
- Children: None
- Siblings: Olivia, Mason
- Grandparents: Sophia, Isabella, Caleb, Logan
- Grandchildren: None
- Piblings: Amelia, Dylan
- Niephlings: None
- Cousins: Sebastian, Stella, Lucas, Penelope


## Future Proof

In order to make the system future proof, I used a variable amount of parents and gender neutral relation terms. This was done to abstractify the entire concept of family to its core component. The important thing is that the family tree represents how a family is connected to eachother and when this system would be furthur developed on, you could replace names with ID's and assign more information to the people in a database. Gender would for example be defined in a seperate dataset to divide functionality from big data.

On this scale, it was not required to abstractify furthur than this, because duplicate names is not an issue yet. When one would implement a system like this in real applications, you add a method to add family members and names would be replaced by id's.